In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
df = pd.read_csv('data.csv', encoding= 'unicode_escape')
df['event_time'] = pd.to_datetime(df['event_time'])
df['user_id'] = df['user_id'].astype(str)
disk_engine = create_engine('sqlite:///marketing_attribution_data.db')

In [3]:
df[df['event_name']=='session_start'].to_sql('session_start', disk_engine, if_exists='replace',index=False)
df[df['event_name']=='purchase'][['event_time',
                                  'session_id',
                                  'event_name',
                                  'user_id',
                                  'transaction_id',
                                  'total_item_quantity',
                                  'purchase_revenue_in_usd']]\
                        .to_sql('purchase', disk_engine, if_exists='replace',index=False)


---
#### Sample Data

In [12]:
#session data
pd.read_sql_query('''
select *
from session_start
limit 5
''',disk_engine)

,event_time,session_id,event_name,medium,campaign_name,source,user_id,transaction_id,total_item_quantity,purchase_revenue_in_usd
0,2021-01-17 10:07:32.602604,2410177724,session_start,organic,(organic),google,1003717.5976366234,None,None,None
1,2021-01-17 15:17:43.775095,8480289037,session_start,(none),(direct),(direct),1007746.000229164,None,None,None
2,2021-01-17 08:23:54.615457,7296599686,session_start,<Other>,<Other>,<Other>,1019199.3353128468,None,None,None
3,2021-01-17 11:09:45.033436,2706513987,session_start,<Other>,<Other>,<Other>,1032465.6915437944,None,None,None
4,2021-01-17 15:27:11.606880,377133069,session_start,referral,(referral),shop.googlemerchandisestore.com,1037187.6235089628,None,None,None


---
#### Transacting User Journey 

In [15]:
pd.read_sql_query('''
select *
from session_start
where user_id = '4670841.6660469435'
limit 5
''',disk_engine)

,event_time,session_id,event_name,medium,campaign_name,source,user_id,transaction_id,total_item_quantity,purchase_revenue_in_usd
0,2021-01-17 23:13:40.193434,4253630718,session_start,referral,(referral),shop.googlemerchandisestore.com,4670841.6660469435,None,None,None
1,2021-01-17 10:09:13.317289,5419224411,session_start,referral,(referral),shop.googlemerchandisestore.com,4670841.6660469435,None,None,None
2,2021-01-16 09:43:12.131002,3851155669,session_start,organic,(organic),google,4670841.6660469435,None,None,None
3,2021-01-19 09:35:54.147124,1198370799,session_start,(none),(direct),(direct),4670841.6660469435,None,None,None
4,2021-01-12 02:22:06.135879,1192057072,session_start,(data deleted),(data deleted),(data deleted),4670841.6660469435,None,None,None


In [14]:
pd.read_sql_query('''
select *
from purchase
where user_id = '4670841.6660469435'
limit 5
''',disk_engine)

,event_time,session_id,event_name,user_id,transaction_id,total_item_quantity,purchase_revenue_in_usd
0,2021-01-17 23:35:07.240500,4253630718,purchase,4670841.6660469435,312823,1.0,69.0


---
#### Non-Transacting User Journey 

In [41]:
pd.read_sql_query('''
select *
from session_start
where user_id = '1019199.3353128468'
limit 100
''',disk_engine)

,event_time,session_id,event_name,medium,campaign_name,source,user_id,transaction_id,total_item_quantity,purchase_revenue_in_usd
0,2021-01-17 08:23:54.615457,7296599686,session_start,<Other>,<Other>,<Other>,1019199.3353128468,None,None,None


In [39]:
pd.read_sql_query('''
select *
from purchase
where user_id = '1019199.3353128468'
''',disk_engine)

,event_time,session_id,event_name,user_id,transaction_id,total_item_quantity,purchase_revenue_in_usd


---
#### Defining Channel based on utm_parameters 

In [56]:
pd.read_sql_query('''
select b.medium,b.campaign_name,b.source,count(*)
from session_start b 
group by 1,2,3
order by 1,2,3
''',disk_engine)

,medium,campaign_name,source,count(*)
0,(data deleted),(data deleted),(data deleted),20662
1,(data deleted),(data deleted),<Other>,376
2,(data deleted),<Other>,(data deleted),435
3,(none),(direct),(direct),82389
4,<Other>,(data deleted),<Other>,1
5,<Other>,<Other>,<Other>,51786
6,<Other>,<Other>,google,1
7,cpc,<Other>,<Other>,1
8,cpc,<Other>,google,15538
9,organic,(organic),<Other>,10084


In [18]:
pd.read_sql_query('''
select b.medium,b.campaign_name,b.source,
case 
    when b.medium = '(data deleted)' then 'Display'
    when b.medium = '<Other>' then 'Display'
    when b.medium = '(none)' then 'Direct'
    when b.medium = 'cpc' then 'SEM'
    when b.medium = 'organic' and source='google' then 'SEO'
    when b.medium = 'organic' and source='<Other>' then 'SEO'
    when b.medium = 'referral' and source='shop.googlemerchandisestore.com' then 'Google Shopping'
    when b.medium = 'referral' then 'Referral Marketing'
    else 'Direct'
    end as channel,
count(*)
from session_start b 
group by 1,2,3,4
order by 1,2,3,4
''',disk_engine)

,medium,campaign_name,source,channel,count(*)
0,(data deleted),(data deleted),(data deleted),Display,20662
1,(data deleted),(data deleted),<Other>,Display,376
2,(data deleted),<Other>,(data deleted),Display,435
3,(none),(direct),(direct),Direct,82389
4,<Other>,(data deleted),<Other>,Display,1
5,<Other>,<Other>,<Other>,Display,51786
6,<Other>,<Other>,google,Display,1
7,cpc,<Other>,<Other>,SEM,1
8,cpc,<Other>,google,SEM,15538
9,organic,(organic),<Other>,SEO,10084


In [17]:
pd.read_sql_query('''
select 
case 
    when b.medium = '(data deleted)' then 'Display'
    when b.medium = '<Other>' then 'Display'
    when b.medium = '(none)' then 'Direct'
    when b.medium = 'cpc' then 'SEM'
    when b.medium = 'organic' and source='google' then 'SEO'
    when b.medium = 'organic' and source='<Other>' then 'SEO'
    when b.medium = 'referral' and source='shop.googlemerchandisestore.com' then 'Google Shopping'
    when b.medium = 'referral' then 'Referral Marketing'
    else 'Direct'
    end as channel,
count(*)
from session_start b 
group by 1
order by 1
''',disk_engine)

,channel,count(*)
0,Direct,82389
1,Display,73261
2,Google Shopping,28060
3,Referral Marketing,34118
4,SEM,15539
5,SEO,121603


---
#### "Marrying" Sales to the Marketing touch

In [50]:
pd.read_sql_query('''
select 
a.*,
b.*
from purchase a 
left join session_start b 
on a.user_id = b.user_id 
    and a.event_time>=b.event_time
    and datetime(a.event_time,'-7 day')<=b.event_time
where a.user_id = '4670841.6660469435'
''',disk_engine)

,event_time,session_id,event_name,user_id,transaction_id,total_item_quantity,purchase_revenue_in_usd,event_time,session_id,event_name,medium,campaign_name,source,user_id,transaction_id,total_item_quantity,purchase_revenue_in_usd
0,2021-01-17 23:35:07.240500,4253630718,purchase,4670841.6660469435,312823,1.0,69.0,2021-01-11 00:43:01.087223,8671526434,session_start,(data deleted),(data deleted),(data deleted),4670841.6660469435,None,None,None
1,2021-01-17 23:35:07.240500,4253630718,purchase,4670841.6660469435,312823,1.0,69.0,2021-01-11 22:57:34.720103,4770818861,session_start,(data deleted),(data deleted),(data deleted),4670841.6660469435,None,None,None
2,2021-01-17 23:35:07.240500,4253630718,purchase,4670841.6660469435,312823,1.0,69.0,2021-01-12 02:22:06.135879,1192057072,session_start,(data deleted),(data deleted),(data deleted),4670841.6660469435,None,None,None
3,2021-01-17 23:35:07.240500,4253630718,purchase,4670841.6660469435,312823,1.0,69.0,2021-01-16 09:43:12.131002,3851155669,session_start,organic,(organic),google,4670841.6660469435,None,None,None
4,2021-01-17 23:35:07.240500,4253630718,purchase,4670841.6660469435,312823,1.0,69.0,2021-01-17 10:09:13.317289,5419224411,session_start,referral,(referral),shop.googlemerchandisestore.com,4670841.6660469435,None,None,None
5,2021-01-17 23:35:07.240500,4253630718,purchase,4670841.6660469435,312823,1.0,69.0,2021-01-17 23:13:40.193434,4253630718,session_start,referral,(referral),shop.googlemerchandisestore.com,4670841.6660469435,None,None,None


In [54]:
pd.read_sql_query('''
select a.*,b.event_time,b.medium,b.campaign_name,b.source
from purchase a 
left join session_start b 
on a.user_id = b.user_id 
    and a.event_time>=b.event_time
    and datetime(a.event_time,'-7 day')<=b.event_time
where a.user_id = '4670841.6660469435'
''',disk_engine)

,event_time,session_id,event_name,user_id,transaction_id,total_item_quantity,purchase_revenue_in_usd,event_time,medium,campaign_name,source
0,2021-01-17 23:35:07.240500,4253630718,purchase,4670841.6660469435,312823,1.0,69.0,2021-01-11 00:43:01.087223,(data deleted),(data deleted),(data deleted)
1,2021-01-17 23:35:07.240500,4253630718,purchase,4670841.6660469435,312823,1.0,69.0,2021-01-11 22:57:34.720103,(data deleted),(data deleted),(data deleted)
2,2021-01-17 23:35:07.240500,4253630718,purchase,4670841.6660469435,312823,1.0,69.0,2021-01-12 02:22:06.135879,(data deleted),(data deleted),(data deleted)
3,2021-01-17 23:35:07.240500,4253630718,purchase,4670841.6660469435,312823,1.0,69.0,2021-01-16 09:43:12.131002,organic,(organic),google
4,2021-01-17 23:35:07.240500,4253630718,purchase,4670841.6660469435,312823,1.0,69.0,2021-01-17 10:09:13.317289,referral,(referral),shop.googlemerchandisestore.com
5,2021-01-17 23:35:07.240500,4253630718,purchase,4670841.6660469435,312823,1.0,69.0,2021-01-17 23:13:40.193434,referral,(referral),shop.googlemerchandisestore.com


In [19]:
pd.read_sql_query('''
select a.*,b.event_time,
    case 
    when b.medium = '(data deleted)' then 'Display'
    when b.medium = '<Other>' then 'Display'
    when b.medium = '(none)' then 'Direct'
    when b.medium = 'cpc' then 'SEM'
    when b.medium = 'organic' and source='google' then 'SEO'
    when b.medium = 'organic' and source='<Other>' then 'SEO'
    when b.medium = 'referral' and source='shop.googlemerchandisestore.com' then 'Google Shopping'
    when b.medium = 'referral' then 'Referral Marketing'
    else 'Direct'
    end as channel
from purchase a 
left join session_start b 
on a.user_id = b.user_id 
    and a.event_time>=b.event_time
    and datetime(a.event_time,'-7 day')<=b.event_time
where a.user_id = '4670841.6660469435'
''',disk_engine)

,event_time,session_id,event_name,user_id,transaction_id,total_item_quantity,purchase_revenue_in_usd,event_time,channel
0,2021-01-17 23:35:07.240500,4253630718,purchase,4670841.6660469435,312823,1.0,69.0,2021-01-11 00:43:01.087223,Display
1,2021-01-17 23:35:07.240500,4253630718,purchase,4670841.6660469435,312823,1.0,69.0,2021-01-11 22:57:34.720103,Display
2,2021-01-17 23:35:07.240500,4253630718,purchase,4670841.6660469435,312823,1.0,69.0,2021-01-12 02:22:06.135879,Display
3,2021-01-17 23:35:07.240500,4253630718,purchase,4670841.6660469435,312823,1.0,69.0,2021-01-16 09:43:12.131002,SEO
4,2021-01-17 23:35:07.240500,4253630718,purchase,4670841.6660469435,312823,1.0,69.0,2021-01-17 10:09:13.317289,Google Shopping
5,2021-01-17 23:35:07.240500,4253630718,purchase,4670841.6660469435,312823,1.0,69.0,2021-01-17 23:13:40.193434,Google Shopping


---
#### Attribution come to action
first touch 7 days

In [21]:
##FIRST TOUCH LAST 7 DAYS
pd.read_sql_query('''
with base as(
select a.*,
b.event_time,
case 
    when b.medium = '(data deleted)' then 'Display'
    when b.medium = '<Other>' then 'Display'
    when b.medium = '(none)' then 'Direct'
    when b.medium = 'cpc' then 'SEM'
    when b.medium = 'organic' and source='google' then 'SEO'
    when b.medium = 'organic' and source='<Other>' then 'SEO'
    when b.medium = 'referral' and source='shop.googlemerchandisestore.com' then 'Google Shopping'
    when b.medium = 'referral' then 'Referral Marketing'
    else 'Direct'
    end as channel,
    row_number() over (partition by a.user_id order by b.event_time) rn
from purchase a 
left join session_start b 
on a.user_id = b.user_id 
    and a.event_time>=b.event_time
    and datetime(a.event_time,'-7 day')<=b.event_time
where a.user_id = '4670841.6660469435'
)
select *
from base
where rn = 1
''',disk_engine)

,event_time,session_id,event_name,user_id,transaction_id,total_item_quantity,purchase_revenue_in_usd,event_time:1,channel,rn
0,2021-01-17 23:35:07.240500,4253630718,purchase,4670841.6660469435,312823,1.0,69.0,2021-01-11 00:43:01.087223,Display,1


In [35]:
##FIRST TOUCH LAST 7 DAYS
pd.read_sql_query('''
with base as(
select a.*,b.event_time as visit_time,case 
    when b.medium = '(data deleted)' then 'Display'
    when b.medium = '<Other>' then 'Display'
    when b.medium = '(none)' then 'Direct'
    when b.medium = 'cpc' then 'SEM'
    when b.medium = 'organic' and source='google' then 'SEO'
    when b.medium = 'organic' and source='<Other>' then 'SEO'
    when b.medium = 'referral' and source='shop.googlemerchandisestore.com' then 'Google Shopping'
    when b.medium = 'referral' then 'Referral Marketing'
    else 'Direct'
    end as channel,
    row_number() over (partition by a.user_id order by b.event_time) rn
from purchase a 
left join session_start b 
on a.user_id = b.user_id 
    and a.event_time>=b.event_time
    and datetime(a.event_time,'-7 day')<=b.event_time
)
select channel, 
strftime('%Y-%m', event_time) as month,
count(*) as n_transactions,
sum(purchase_revenue_in_usd) as total_revenue


from base
where rn = 1
group by 1,2
order by 2,1
''',disk_engine)

,channel,month,n_transactions,total_revenue
0,Direct,2020-11,369,26532.0
1,Display,2020-11,340,22115.0
2,Google Shopping,2020-11,151,10716.0
3,Referral Marketing,2020-11,135,10888.0
4,SEM,2020-11,67,4679.0
5,SEO,2020-11,468,34409.0
6,Direct,2020-12,422,26872.0
7,Display,2020-12,397,27173.0
8,Google Shopping,2020-12,160,12171.0
9,Referral Marketing,2020-12,177,11082.0


In [38]:
##Session Data
pd.read_sql_query('''

select case 
    when medium = '(data deleted)' then 'Display'
    when medium = '<Other>' then 'Display'
    when medium = '(none)' then 'Direct'
    when medium = 'cpc' then 'SEM'
    when medium = 'organic' and source='google' then 'SEO'
    when medium = 'organic' and source='<Other>' then 'SEO'
    when medium = 'referral' and source='shop.googlemerchandisestore.com' then 'Google Shopping'
    when medium = 'referral' then 'Referral Marketing'
    else 'Direct'
    end as channel,
strftime('%Y-%m', event_time) as month,
count(*) as n_sessions


from session_start
group by 1,2
order by 2,1
''',disk_engine)

,channel,month,n_sessions
0,Direct,2020-11,24651
1,Display,2020-11,22552
2,Google Shopping,2020-11,8986
3,Referral Marketing,2020-11,10338
4,SEM,2020-11,4428
5,SEO,2020-11,35630
6,Direct,2020-12,30516
7,Display,2020-12,27035
8,Google Shopping,2020-12,10428
9,Referral Marketing,2020-12,12664


In [42]:
##FIRST TOUCH LAST 7 DAYS
pd.read_sql_query('''
with base as(
select a.*,b.event_time as visit_time,case 
    when b.medium = '(data deleted)' then 'Display'
    when b.medium = '<Other>' then 'Display'
    when b.medium = '(none)' then 'Direct'
    when b.medium = 'cpc' then 'SEM'
    when b.medium = 'organic' and source='google' then 'SEO'
    when b.medium = 'organic' and source='<Other>' then 'SEO'
    when b.medium = 'referral' and source='shop.googlemerchandisestore.com' then 'Google Shopping'
    when b.medium = 'referral' then 'Referral Marketing'
    else 'Direct'
    end as channel,
    row_number() over (partition by a.user_id order by b.event_time) rn
from purchase a 
left join session_start b 
on a.user_id = b.user_id 
    and a.event_time>=b.event_time
    and datetime(a.event_time,'-7 day')<=b.event_time
),
attribution_summary as (
select channel, 
strftime('%Y-%m', event_time) as month,
count(*) as n_transactions,
sum(purchase_revenue_in_usd) as total_revenue


from base
where rn = 1
group by 1,2
order by 2,1),
session_summary as (
select case 
    when medium = '(data deleted)' then 'Display'
    when medium = '<Other>' then 'Display'
    when medium = '(none)' then 'Direct'
    when medium = 'cpc' then 'SEM'
    when medium = 'organic' and source='google' then 'SEO'
    when medium = 'organic' and source='<Other>' then 'SEO'
    when medium = 'referral' and source='shop.googlemerchandisestore.com' then 'Google Shopping'
    when medium = 'referral' then 'Referral Marketing'
    else 'Direct'
    end as channel,
strftime('%Y-%m', event_time) as month,
count(*) as n_sessions


from session_start
group by 1,2
order by 2,1
)
select a.channel,a.month,n_sessions,n_transactions,round(cast(n_transactions as float)/n_sessions,3) as channel_cr
from session_summary a 
left join attribution_summary b 
on a.channel=b.channel and a.month=b.month

''',disk_engine)

,channel,month,n_sessions,n_transactions,channel_cr
0,Direct,2020-11,24651,369,0.015
1,Display,2020-11,22552,340,0.015
2,Google Shopping,2020-11,8986,151,0.017
3,Referral Marketing,2020-11,10338,135,0.013
4,SEM,2020-11,4428,67,0.015
5,SEO,2020-11,35630,468,0.013
6,Direct,2020-12,30516,422,0.014
7,Display,2020-12,27035,397,0.015
8,Google Shopping,2020-12,10428,160,0.015
9,Referral Marketing,2020-12,12664,177,0.014


---
Last touch attribution

In [43]:
##LAST TOUCH LAST 7 DAYS
pd.read_sql_query('''
with base as(
select a.*,
b.event_time,
case 
    when b.medium = '(data deleted)' then 'Display'
    when b.medium = '<Other>' then 'Display'
    when b.medium = '(none)' then 'Direct'
    when b.medium = 'cpc' then 'SEM'
    when b.medium = 'organic' and source='google' then 'SEO'
    when b.medium = 'organic' and source='<Other>' then 'SEO'
    when b.medium = 'referral' and source='shop.googlemerchandisestore.com' then 'Google Shopping'
    when b.medium = 'referral' then 'Referral Marketing'
    else 'Direct'
    end as channel,
    row_number() over (partition by a.user_id order by b.event_time desc) rn
from purchase a 
left join session_start b 
on a.user_id = b.user_id 
    and a.event_time>=b.event_time
    and datetime(a.event_time,'-7 day')<=b.event_time
where a.user_id = '4670841.6660469435'
)
select *
from base
where rn = 1
''',disk_engine)

,event_time,session_id,event_name,user_id,transaction_id,total_item_quantity,purchase_revenue_in_usd,event_time:1,channel,rn
0,2021-01-17 23:35:07.240500,4253630718,purchase,4670841.6660469435,312823,1.0,69.0,2021-01-17 23:13:40.193434,Google Shopping,1


In [44]:
##LAST TOUCH
pd.read_sql_query('''
with base as(
select a.*,b.event_time as visit_time,case 
    when b.medium = '(data deleted)' then 'Display'
    when b.medium = '<Other>' then 'Display'
    when b.medium = '(none)' then 'Direct'
    when b.medium = 'cpc' then 'SEM'
    when b.medium = 'organic' and source='google' then 'SEO'
    when b.medium = 'organic' and source='<Other>' then 'SEO'
    when b.medium = 'referral' and source='shop.googlemerchandisestore.com' then 'Google Shopping'
    when b.medium = 'referral' then 'Referral Marketing'
    else 'Direct'
    end as channel,
    row_number() over (partition by a.user_id order by b.event_time DESC) rn
from purchase a 
left join session_start b 
on a.user_id = b.user_id 
    and a.event_time>=b.event_time
    and datetime(a.event_time,'-7 day')<=b.event_time
),
attribution_summary as (
select channel, 
strftime('%Y-%m', event_time) as month,
count(*) as n_transactions,
sum(purchase_revenue_in_usd) as total_revenue


from base
where rn = 1
group by 1,2
order by 2,1),
session_summary as (
select case 
    when medium = '(data deleted)' then 'Display'
    when medium = '<Other>' then 'Display'
    when medium = '(none)' then 'Direct'
    when medium = 'cpc' then 'SEM'
    when medium = 'organic' and source='google' then 'SEO'
    when medium = 'organic' and source='<Other>' then 'SEO'
    when medium = 'referral' and source='shop.googlemerchandisestore.com' then 'Google Shopping'
    when medium = 'referral' then 'Referral Marketing'
    else 'Direct'
    end as channel,
strftime('%Y-%m', event_time) as month,
count(*) as n_sessions


from session_start
group by 1,2
order by 2,1
)
select a.channel,a.month,n_sessions,n_transactions,round(cast(n_transactions as float)/n_sessions,3) as channel_cr
from session_summary a 
left join attribution_summary b 
on a.channel=b.channel and a.month=b.month

''',disk_engine)

,channel,month,n_sessions,n_transactions,channel_cr
0,Direct,2020-11,24651,329,0.013
1,Display,2020-11,22552,360,0.016
2,Google Shopping,2020-11,8986,173,0.019
3,Referral Marketing,2020-11,10338,147,0.014
4,SEM,2020-11,4428,47,0.011
5,SEO,2020-11,35630,367,0.010
6,Direct,2020-12,30516,452,0.015
7,Display,2020-12,27035,466,0.017
8,Google Shopping,2020-12,10428,222,0.021
9,Referral Marketing,2020-12,12664,174,0.014


---
Multi touch attribution: Equal Distribution

In [47]:
##Equally Distributed
pd.read_sql_query('''
with base as(
select a.*,b.event_time,case 
    when medium = '(data deleted)' then 'Display'
    when medium = '<Other>' then 'Display'
    when medium = '(none)' then 'Direct'
    when medium = 'cpc' then 'SEM'
    when medium = 'organic' and source='google' then 'SEO'
    when medium = 'organic' and source='<Other>' then 'SEO'
    when medium = 'referral' and source='shop.googlemerchandisestore.com' then 'Google Shopping'
    when medium = 'referral' then 'Referral Marketing'
    else 'Direct'
    end as channel,
    count(*) over (partition by a.user_id) n_touches
from purchase a 
left join session_start b 
on a.user_id = b.user_id 
    and a.event_time>=b.event_time
    and datetime(a.event_time,'-7 day')<=b.event_time
where a.user_id = '4670841.6660469435'
)
select *,
        purchase_revenue_in_usd/n_touches,
        1.0/n_touches as distributed_transactions

from base

''',disk_engine)

,event_time,session_id,event_name,user_id,transaction_id,total_item_quantity,purchase_revenue_in_usd,event_time:1,channel,n_touches,purchase_revenue_in_usd/n_touches,distributed_transactions
0,2021-01-17 23:35:07.240500,4253630718,purchase,4670841.6660469435,312823,1.0,69.0,2021-01-11 00:43:01.087223,Display,6,11.5,0.166667
1,2021-01-17 23:35:07.240500,4253630718,purchase,4670841.6660469435,312823,1.0,69.0,2021-01-11 22:57:34.720103,Display,6,11.5,0.166667
2,2021-01-17 23:35:07.240500,4253630718,purchase,4670841.6660469435,312823,1.0,69.0,2021-01-12 02:22:06.135879,Display,6,11.5,0.166667
3,2021-01-17 23:35:07.240500,4253630718,purchase,4670841.6660469435,312823,1.0,69.0,2021-01-16 09:43:12.131002,SEO,6,11.5,0.166667
4,2021-01-17 23:35:07.240500,4253630718,purchase,4670841.6660469435,312823,1.0,69.0,2021-01-17 10:09:13.317289,Google Shopping,6,11.5,0.166667
5,2021-01-17 23:35:07.240500,4253630718,purchase,4670841.6660469435,312823,1.0,69.0,2021-01-17 23:13:40.193434,Google Shopping,6,11.5,0.166667


In [49]:
##LAST TOUCH LAST Equally Distributed
pd.read_sql_query('''
with base as(
select a.*,b.event_time,case 
    when medium = '(data deleted)' then 'Display'
    when medium = '<Other>' then 'Display'
    when medium = '(none)' then 'Direct'
    when medium = 'cpc' then 'SEM'
    when medium = 'organic' and source='google' then 'SEO'
    when medium = 'organic' and source='<Other>' then 'SEO'
    when medium = 'referral' and source='shop.googlemerchandisestore.com' then 'Google Shopping'
    when medium = 'referral' then 'Referral Marketing'
    else 'Direct'
    end as channel,
    count(*) over (partition by a.user_id) n_touches
from purchase a 
left join session_start b 
on a.user_id = b.user_id 
    and a.event_time>=b.event_time
    and datetime(a.event_time,'-7 day')<=b.event_time

)
select channel, 
strftime('%Y-%m', event_time) as month,
sum(1.0/n_touches) as n_transactions,
sum(purchase_revenue_in_usd/n_touches) as total_revenue
from base
group by 1,2
order by 2,1
''',disk_engine)

,channel,month,n_transactions,total_revenue
0,Direct,2020-11,353.925205,25425.397015
1,Display,2020-11,352.542733,23317.140201
2,Google Shopping,2020-11,164.300168,11999.109214
3,Referral Marketing,2020-11,139.504545,10737.571920
4,SEM,2020-11,52.997136,3267.663503
5,SEO,2020-11,417.547006,30464.471782
6,Direct,2020-12,438.356014,29469.605076
7,Display,2020-12,443.291221,29392.912041
8,Google Shopping,2020-12,196.213109,14390.507465
9,Referral Marketing,2020-12,175.075916,12003.105140


In [51]:
pd.read_sql_query('''
with base as(
select a.*,b.event_time,case 
    when medium = '(data deleted)' then 'Display'
    when medium = '<Other>' then 'Display'
    when medium = '(none)' then 'Direct'
    when medium = 'cpc' then 'SEM'
    when medium = 'organic' and source='google' then 'SEO'
    when medium = 'organic' and source='<Other>' then 'SEO'
    when medium = 'referral' and source='shop.googlemerchandisestore.com' then 'Google Shopping'
    when medium = 'referral' then 'Referral Marketing'
    else 'Direct'
    end as channel,
    count(*) over (partition by a.user_id) n_touches
from purchase a 
left join session_start b 
on a.user_id = b.user_id 
    and a.event_time>=b.event_time
    and datetime(a.event_time,'-7 day')<=b.event_time
),
attribution_summary as (
select channel, 
strftime('%Y-%m', event_time) as month,
sum(1.0/n_touches) as n_transactions,
sum(purchase_revenue_in_usd/n_touches) as total_revenue
from base
group by 1,2
order by 2,1),

session_summary as (
select case 
    when medium = '(data deleted)' then 'Display'
    when medium = '<Other>' then 'Display'
    when medium = '(none)' then 'Direct'
    when medium = 'cpc' then 'SEM'
    when medium = 'organic' and source='google' then 'SEO'
    when medium = 'organic' and source='<Other>' then 'SEO'
    when medium = 'referral' and source='shop.googlemerchandisestore.com' then 'Google Shopping'
    when medium = 'referral' then 'Referral Marketing'
    else 'Direct'
    end as channel,
strftime('%Y-%m', event_time) as month,
count(*) as n_sessions


from session_start
group by 1,2
order by 2,1
)
select a.channel,a.month,n_sessions,n_transactions,round(cast(n_transactions as float)/n_sessions,3) as channel_cr
from session_summary a 
left join attribution_summary b 
on a.channel=b.channel and a.month=b.month

''',disk_engine)

,channel,month,n_sessions,n_transactions,channel_cr
0,Direct,2020-11,24651,353.925205,0.014
1,Display,2020-11,22552,352.542733,0.016
2,Google Shopping,2020-11,8986,164.300168,0.018
3,Referral Marketing,2020-11,10338,139.504545,0.013
4,SEM,2020-11,4428,52.997136,0.012
5,SEO,2020-11,35630,417.547006,0.012
6,Direct,2020-12,30516,438.356014,0.014
7,Display,2020-12,27035,443.291221,0.016
8,Google Shopping,2020-12,10428,196.213109,0.019
9,Referral Marketing,2020-12,12664,175.075916,0.014


---
U Shape Attribution

In [55]:
##U-Shape attribution
pd.read_sql_query('''
with base as(
select a.*,b.event_time,case 
    when medium = '(data deleted)' then 'Display'
    when medium = '<Other>' then 'Display'
    when medium = '(none)' then 'Direct'
    when medium = 'cpc' then 'SEM'
    when medium = 'organic' and source='google' then 'SEO'
    when medium = 'organic' and source='<Other>' then 'SEO'
    when medium = 'referral' and source='shop.googlemerchandisestore.com' then 'Google Shopping'
    when medium = 'referral' then 'Referral Marketing'
    else 'Direct'
    end as channel,
    count(*) over (partition by a.user_id) n_touches,
    row_number() over (partition by a.user_id order by b.event_time desc) rn_desc,
    row_number() over (partition by a.user_id order by b.event_time asc) rn_asc
from purchase a 
left join session_start b 
on a.user_id = b.user_id 
    and a.event_time>=b.event_time
    and datetime(a.event_time,'-7 day')<=b.event_time
where a.user_id = '4670841.6660469435'
),
create_weight as (
select *,
        case when n_touches= 1 then 1
             when n_touches= 2 then 0.5
             when n_touches> 2 and rn_desc=1 then 0.4
             when n_touches> 2 and rn_asc=1 then 0.4
             else 0.2/(n_touches-2)
        end as         
        weight
from base
)
select *,weight*purchase_revenue_in_usd as revenue_split,
        weight as n_transactions
from create_weight

''',disk_engine)

,event_time,session_id,event_name,user_id,transaction_id,total_item_quantity,purchase_revenue_in_usd,event_time:1,channel,n_touches,rn_desc,rn_asc,weight,revenue_split,n_transactions
0,2021-01-17 23:35:07.240500,4253630718,purchase,4670841.6660469435,312823,1.0,69.0,2021-01-17 23:13:40.193434,Google Shopping,6,1,6,0.40,27.60,0.40
1,2021-01-17 23:35:07.240500,4253630718,purchase,4670841.6660469435,312823,1.0,69.0,2021-01-17 10:09:13.317289,Google Shopping,6,2,5,0.05,3.45,0.05
2,2021-01-17 23:35:07.240500,4253630718,purchase,4670841.6660469435,312823,1.0,69.0,2021-01-16 09:43:12.131002,SEO,6,3,4,0.05,3.45,0.05
3,2021-01-17 23:35:07.240500,4253630718,purchase,4670841.6660469435,312823,1.0,69.0,2021-01-12 02:22:06.135879,Display,6,4,3,0.05,3.45,0.05
4,2021-01-17 23:35:07.240500,4253630718,purchase,4670841.6660469435,312823,1.0,69.0,2021-01-11 22:57:34.720103,Display,6,5,2,0.05,3.45,0.05
5,2021-01-17 23:35:07.240500,4253630718,purchase,4670841.6660469435,312823,1.0,69.0,2021-01-11 00:43:01.087223,Display,6,6,1,0.40,27.60,0.40


In [56]:
pd.read_sql_query('''
with base as(
select a.*,b.event_time,case 
    when medium = '(data deleted)' then 'Display'
    when medium = '<Other>' then 'Display'
    when medium = '(none)' then 'Direct'
    when medium = 'cpc' then 'SEM'
    when medium = 'organic' and source='google' then 'SEO'
    when medium = 'organic' and source='<Other>' then 'SEO'
    when medium = 'referral' and source='shop.googlemerchandisestore.com' then 'Google Shopping'
    when medium = 'referral' then 'Referral Marketing'
    else 'Direct'
    end as channel,
    count(*) over (partition by a.user_id) n_touches,
    row_number() over (partition by a.user_id order by b.event_time desc) rn_desc,
    row_number() over (partition by a.user_id order by b.event_time asc) rn_asc
from purchase a 
left join session_start b 
on a.user_id = b.user_id 
    and a.event_time>=b.event_time
    and datetime(a.event_time,'-7 day')<=b.event_time
),
create_weight as (
select *,
        case when n_touches= 1 then 1
             when n_touches= 2 then 0.5
             when n_touches> 2 and rn_desc=1 then 0.4
             when n_touches> 2 and rn_asc=1 then 0.4
             else 0.2/(n_touches-2)
        end as         
        weight
from base
),
attribution_summary as (
select channel, 
strftime('%Y-%m', event_time) as month,
sum(weight) as n_transactions

from create_weight
group by 1,2
order by 2,1),

session_summary as (
select case 
    when medium = '(data deleted)' then 'Display'
    when medium = '<Other>' then 'Display'
    when medium = '(none)' then 'Direct'
    when medium = 'cpc' then 'SEM'
    when medium = 'organic' and source='google' then 'SEO'
    when medium = 'organic' and source='<Other>' then 'SEO'
    when medium = 'referral' and source='shop.googlemerchandisestore.com' then 'Google Shopping'
    when medium = 'referral' then 'Referral Marketing'
    else 'Direct'
    end as channel,
strftime('%Y-%m', event_time) as month,
count(*) as n_sessions


from session_start
group by 1,2
order by 2,1
)
select a.channel,a.month,n_sessions,n_transactions,round(cast(n_transactions as float)/n_sessions,3) as channel_cr
from session_summary a 
left join attribution_summary b 
on a.channel=b.channel and a.month=b.month

''',disk_engine)

,channel,month,n_sessions,n_transactions,channel_cr
0,Direct,2020-11,24651,349.925011,0.014
1,Display,2020-11,22552,348.635797,0.015
2,Google Shopping,2020-11,8986,162.491851,0.018
3,Referral Marketing,2020-11,10338,140.631377,0.014
4,SEM,2020-11,4428,55.473225,0.013
5,SEO,2020-11,35630,415.850683,0.012
6,Direct,2020-12,30516,437.165398,0.014
7,Display,2020-12,27035,436.074857,0.016
8,Google Shopping,2020-12,10428,193.374812,0.019
9,Referral Marketing,2020-12,12664,174.030628,0.014
